# Grant.gov

## Initialize

In [ ]:
import bs4 as bs
import requests 

In [ ]:
#tutourial https://medium.com/ymedialabs-innovation/web-scraping-using-beautiful-soup-and-selenium-for-dynamic-page-2f8ad15efe25
#initialize
URL = "https://www.grants.gov/web/grants/search-grants.html"
r = requests.get(URL) 
  
soup = bs.BeautifulSoup(r.content, 'html5lib')
print(soup.prettify()) 

## Test Filter 9/21/2020

In [ ]:
##################################### BEGIN TEST SECTION #####################################################
## install packages
# change this binary parameter below to match your chrome version
# stable version 85 recommended
# pip install chromedriver-binary==85.0.4183.87
# tutourial for iframe: https://www.techbeamers.com/switch-between-iframes-selenium-python/
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver import chrome
import time

# use selenium to apply filters
driver = webdriver.Chrome()
driver.get("https://www.grants.gov/web/grants/search-grants.html")
# action delay
driver.implicitly_wait(5)
#check if 
tags = driver.find_elements_by_tag_name('iframe')
tags
# if tags non empty, html contains iframe

#html example
#<input type="checkbox" class="formfield" onclick="updateSearchResults( this );" name="fundingInstruments" value="CA" id="CA" title="Cooperative Agreement (241)">

In [ ]:
iframe = driver.find_elements_by_tag_name('iframe')[0]

In [ ]:
driver.switch_to.frame(iframe)

In [ ]:
test = driver.find_element_by_id('CA')

In [ ]:
#simulate a click
test.click()
#log 9/22/2020 success
######################################### END TEST SECTION ###########################################################

## Apply Filters 9/22/2020 START HERE

In [1]:
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver import chrome
import time

driver = webdriver.Chrome()
driver.get("https://www.grants.gov/web/grants/search-grants.html")
driver.implicitly_wait(5)
tags = driver.find_elements_by_tag_name('iframe')
tags
time.sleep(3)

In [2]:
iframe = driver.find_elements_by_tag_name('iframe')[0]
driver.switch_to.frame(iframe)

In [3]:
# clicking simulation tailored to each site
# have to manually choose tags based on needs

################################### FILTER TAGS #############################
#Opportunity Type
    # Separate forecasted and posted
    # Forecasted: uncheck forcasted
forecasted = driver.find_element_by_id('forecasted')
forecasted.click()

    # Posted
#posted = driver.find_element_by_id('posted')
#posted.click()

## Page refresh: wait for DOM to reattach reference
time.sleep(1)

In [4]:
#Funding Instrument Type
    #Grant
G = driver.find_element_by_id('G')
G.click()
time.sleep(1)

In [5]:
#Eligibility

# Nonprofits having a 501(c)(3) status with the IRS, other than institutions of higher education
irs = driver.find_element_by_id('12')
irs.click()
time.sleep(1)

# Nonprofits that do not have a 501(c)(3) status with the IRS, other than institutions of higher education
nirs = driver.find_element_by_id('13')
nirs.click()
time.sleep(1)

# Unrestricted (i.e., open to any type of entity above), subject to any clarification in text field entitled
# 'Additional Information on Eligibility' 
nirs = driver.find_element_by_id('99')
nirs.click()
time.sleep(2)

In [6]:
#Category
#education
edu = driver.find_element_by_id('ED')
edu.click()
time.sleep(1)
#science and technology
st = driver.find_element_by_id('ST')
st.click()
time.sleep(1)
#Employment, Labor and Training
elt = driver.find_element_by_id('ELT')
elt.click()
time.sleep(1)
#Environment
env = driver.find_element_by_id('ENV')
env.click()
time.sleep(1)

## Scraping 10/01/2020-10/03/2020

In [24]:
import bs4 as bs
name = []

#Funding Opportunity Number
opnum = []
#eligibility
el = []
#agency name
agn = []
#description
des = []

def collect():

    driver.switch_to.default_content()
    try:
        iframe = driver.find_elements_by_tag_name('iframe')[0]
        driver.switch_to.frame(iframe)
    except:
        pass
    hasPage = True
    while(hasPage):
        for i in range(2,26):
            #load page with scraper
            driver.find_element_by_css_selector('#searchResultsDiv > table.grid > tbody > tr:nth-child('+str(i)+') > td:nth-child(1) > a').click()
            time.sleep(2)
            s = bs.BeautifulSoup(driver.page_source,'lxml')

            grantInfo = s.find('div',attrs={"id":"synopsisDetailsGeneralInfoTableLeft"})
            rows = grantInfo.find_all('tr')
            driver.implicitly_wait(1)
            #store general info
            # opportunity number
            try:
                opnum.append(rows[1].find('td').text)
            except:
                #wait longer
                time.sleep(5)
                
                s = bs.BeautifulSoup(driver.page_source,'lxml')

                grantInfo = s.find('div',attrs={"id":"synopsisDetailsGeneralInfoTableLeft"})
                rows = grantInfo.find_all('tr')
                driver.implicitly_wait(1)
                opnum.append(rows[1].find('td').text)
                
            #Funding Opportunity Title
            name.append(rows[2].find('td').text)

            #Grant eligibility
            grantInfo = s.find('div',attrs={"id":"synopsisDetailsEligibilityTable"})
            rows = grantInfo.find_all('tr')
            el.append(rows[0].find('td').text)

            grantInfo = s.find('div',attrs={"id":"synopsisDetailsAdditionalInfoTable"})
            rows = grantInfo.find_all('tr')
            #agency name
            agn.append(rows[0].find('td').text)
            #description
            des.append(rows[1].find('td').text)
            time.sleep(2)
            #return to previous page
            driver.execute_script("window.history.go(-1)")
            driver.implicitly_wait(1)
        try:
            driver.find_element_by_css_selector("[title*='Click Next Page']").click()
        except:
            print("no pages left")
            return
        


In [25]:
collect()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#searchResultsDiv > table.grid > tbody > tr:nth-child(18) > td:nth-child(1) > a"}
  (Session info: chrome=85.0.4183.121)


In [26]:
#store in dictionary
data = {"Name":name,
       "Opportunity Number":opnum,
       "Eligibility":el,
       "Agency Name": agn,
       "Description":des}

In [27]:
#driver.find_element_by_css_selector("[title*='Click Next Page']").click()

In [28]:
import pandas as pd
df = pd.DataFrame(data)
df.to_csv('grantGov_raw.txt',index=True)

In [29]:
#searchResultsDiv > table.grid > tbody > tr:nth-child(26) > td:nth-child(1) > a
df

,Name,Opportunity Number,Eligibility,Agency Name,Description
0,Transitions to Excellence in Molecular and Cel...,21-508,"Unrestricted (i.e., open to any type of entity...",National Science Foundation,The Division of Molecular and Cellular Bioscie...
1,Plant Genome Research Program,21-507,"Unrestricted (i.e., open to any type of entity...",National Science Foundation,\n\n\n\nThe Plant Genome Research Program (PGR...
2,Division of Integrative Organismal Systems Cor...,21-506,"Unrestricted (i.e., open to any type of entity...",National Science Foundation,\n\nThe Division of Integrative Organismal Sys...
3,Algebra and Number Theory,PD-20-1264,"Unrestricted (i.e., open to any type of entity...",National Science Foundation,The Algebra and Number Theory program supports...
4,Successor-in-Interest (Type 6 Parent Clinical ...,PA-20-275,Public housing authorities/Indian housing auth...,National Institutes of Health,The National Institutes of Health (NIH) hereby...
...,...,...,...,...,...
539,Ocean Technology and Interdisciplinary Coordin...,PD-98-1680,"Unrestricted (i.e., open to any type of entity...",National Science Foundation,The Oceanographic Technology and Interdiscipli...
540,Biological Oceanography,PD-98-1650,"Unrestricted (i.e., open to any type of entity...",National Science Foundation,The Biological Oceanography Program supports f...
541,Biological Anthropology,PD-98-1392,"Unrestricted (i.e., open to any type of entity...",National Science Foundation,The Biological Anthropology Program supports b...
542,Archaeology and Archaeometry,PD-98-1391,"Unrestricted (i.e., open to any type of entity...",National Science Foundation,The goal of the Archaeology Program is to fund...


# Foundation Directory Online

In [ ]:
#reads a .txt in current directory
#tutorial: https://www3.ntu.edu.sg/home/ehchua/programming/webprogramming/Python_FileText.html
login = open('loginfo_mia.txt','r')
email = login.readline().rstrip()
uname = login.readline().rstrip()
passw = login.readline().rstrip()
login.close()

#initialize packages
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver import chrome
import time

driver = webdriver.Chrome()
#replace this with your institution's link to Candid database
#link = 'https://fconline-foundationcenter-org.ezproxy.lib.vt.edu/search/'


################################### Begin 2FA and EZproxy bypass ####################################
link = 'https://www.google.com/'
#visit site
driver.get(link)
time.sleep(5)
authGoog = driver.find_elements_by_xpath('//*[@id="gb_70"]')[0]
authGoog.click()
time.sleep(3)
authGoog = driver.find_elements_by_xpath('//*[@id="identifierId"]')[0]
authGoog.send_keys(email)
time.sleep(1)
authGoog = driver.find_elements_by_xpath('//*[@id="identifierNext"]/div/button/div[2]')[0]
authGoog.click()
time.sleep(5)
username = driver.find_element_by_id('username')
username.send_keys(uname)
password = driver.find_element_by_id('password')
password.send_keys(passw)
loginButt=driver.find_element_by_name('_eventId_proceed')
loginButt.click()
# wait for 2-factor
time.sleep(5)
iframe = driver.find_elements_by_tag_name('iframe')[0]
driver.switch_to.frame(iframe)
auth = driver.find_elements_by_xpath('//*[@id="auth_methods"]/fieldset/div[1]/button')
auth[0].click()
time.sleep(6)

# Performs mouse move action onto the element
# simulate human mouse movement
webdriver.ActionChains(driver).move_by_offset(100,100).perform()
webdriver.ActionChains(driver).move_by_offset(98,99).perform()
webdriver.ActionChains(driver).move_by_offset(90,89).perform()
time.sleep(1)


# element = driver.find_elements_by_xpath('//*[@id="view_container"]/div/div/div[2]/div/div[2]/div/div[1]/div/div/button')
# action.move_to_element(element) 
# time.sleep(1)

In [ ]:
# must interact with window. Anti bot measures present
confirm = driver.find_element_by_css_selector('#view_container > div > div > div.pwWryf.bxPAYd > div > div.zQJV3 > div > div.qhFLie > div > div > button')
confirm.click()

## FDO Filtering and Scraping 10/5/2020-10/6/2020

In [ ]:
#login thru vtlib
lk = 'https://guides.lib.vt.edu/find/grants'
driver.get(lk)
time.sleep(5)
FDO = driver.find_element_by_css_selector('#s-lg-content-303847 > a')
FDO.click()
time.sleep(3)

In [ ]:
#Start filtering
FDO = driver.find_element_by_css_selector('#js-toggle-search')
FDO.click()
time.sleep(1)

In [ ]:
#first filter
FDO = driver.find_element_by_css_selector('#subject_area-tokenfield')
FDO.click()
#education
FDO = driver.find_element_by_css_selector('#input-subject_area-SB')
FDO.click()

In [ ]:
#Science>
FDO = driver.find_element_by_css_selector('#accordionundefined > div > div:nth-child(29) > span')
FDO.click()
driver.implicitly_wait(1)


In [ ]:
#technology>
FDO = driver.find_element_by_css_selector('#accordion1114 > div > div:nth-child(9) > span')
driver.implicitly_wait(1)
FDO.click()

In [ ]:
#>Computer Science
FDO = driver.find_element_by_css_selector('#input-subject_area-SF0203')
driver.implicitly_wait(1)
FDO.click()
#>Robotics
FDO = driver.find_element_by_css_selector('#input-subject_area-SF0205')
driver.implicitly_wait(1)
FDO.click()
#Environment>
#Environmental education
driver.implicitly_wait(1)

In [ ]:
#collapse Subject Area
FDO = driver.find_element_by_css_selector('#accordion1 > div > button > i')
FDO.click()

In [ ]:
# FDO = driver.find_element_by_css_selector('#main-search-form > div:nth-child(1) > div:nth-child(5) > div > div > span > input')
# FDO.send_keys('dallas')
# time.sleep(1)
# loc = driver.find_element_by_css_selector('#listbox_organization_location > div > div > div:nth-child(3) > label')
# loc.click()
# time.sleep(1)

In [ ]:
# #collapse Subject Area
# FDO = driver.find_element_by_css_selector('#listbox_organization_location > div > button > i')
# FDO.click()

In [ ]:
#Search button
FDO = driver.find_element_by_css_selector('#prepare-elastic-input')
FDO.click()
time.sleep(1)
FDO = driver.find_element_by_css_selector('#grants-view-all > a')
FDO.click()

## Test Chrome Session

In [ ]:
import os
print(os.getcwd())

In [ ]:
confirm = driver.find_element_by_css_selector('#view_container > div > div > div.pwWryf.bxPAYd > div > div.zQJV3 > div > div.qhFLie > div > div > button')
confirm.click()

#main-search-form > div:nth-child(1) > div:nth-child(5) > div > div > span > input

## Additional Test Section

In [ ]:
auth = driver.find_elements_by_xpath('//*[@id="auth_methods"]/fieldset/div[1]/button')

In [ ]:
name = []
opnum = []
#Funding Opportunity Number
fo = []
#eligibility
el = []
#agency name
agn = []
#description
des = []

deet = []

s = bs.BeautifulSoup(driver.page_source,'lxml')
grantInfo = s.find('div',attrs={"id":"synopsisDetailsGeneralInfoTableLeft"})

rows = grantInfo.find_all('tr')

#store general info
# opportunity number
try:
    opnum.append(rows[1].find('td').text)
except:
    #last page
    pass
#Funding Opportunity Title
name.append(rows[2].find('td').text)

#Grant eligibility
grantInfo = s.find('div',attrs={"id":"synopsisDetailsEligibilityTable"})
rows = grantInfo.find_all('tr')
el.append(rows[0].find('td').text)

grantInfo = s.find('div',attrs={"id":"synopsisDetailsAdditionalInfoTable"})
rows = grantInfo.find_all('tr')
#agency name
agn.append(rows[0].find('td').text)
#description
des.append(rows[1].find('td').text)
time.sleep(2)

In [ ]:
name

In [ ]:
opnum

In [ ]:
fo

In [ ]:
el

In [ ]:
agn

In [ ]:
des

In [ ]:
deet

In [ ]:
driver.execute_script("window.history.go(-1)")

In [ ]:
driver.find_element_by_css_selector('#searchResultsDiv > table.grid > tbody > tr:nth-child(2) > td:nth-child(1) > a').click()
s = bs.BeautifulSoup(driver.page_source,'lxml')

In [ ]:
i=str(19)
driver.find_element_by_css_selector('#searchResultsDiv > table.grid > tbody > tr:nth-child('+i+') > td:nth-child(1) > a').click()

In [ ]:
i=str(18)
driver.find_element_by_css_selector('#searchResultsDiv > table.grid > tbody > tr:nth-child('+str(i)+') > td:nth-child(1) > a').click()
#searchResultsDiv > table.grid > tbody > tr:nth-child(2) > td:nth-child(1) > a
#searchResultsDiv > table.grid > tbody > tr:nth-child(3) > td:nth-child(1) > a
driver.implicitly_wait(1)
s = bs.BeautifulSoup(driver.page_source,'lxml')
time.sleep(1)

grantInfo = s.find('div',attrs={"id":"synopsisDetailsGeneralInfoTableLeft"})
time.sleep(2)
rows = grantInfo.find_all('tr')
driver.implicitly_wait(1)
#store general info
# opportunity number
#             try:
opnum.append(rows[1].find('td').text)

In [ ]:
grantInfo

In [ ]:
rows = grantInfo.find_all('tr')

In [ ]:
rows

In [ ]:
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver import chrome
import time

driver = webdriver.Chrome()
driver.switch_to.default_content()
iframe = driver.find_elements_by_tag_name('iframe')[0]
driver.switch_to.frame(iframe)

i=str(2)
driver.find_element_by_css_selector('#searchResultsDiv > table.grid > tbody > tr:nth-child('+str(i)+') > td:nth-child(1) > a').click()
s = bs.BeautifulSoup(driver.page_source,'lxml')
time.sleep(1)

grantInfo = s.find('div',attrs={"id":"synopsisDetailsGeneralInfoTableLeft"})
# time.sleep(2)
rows = grantInfo.find_all('tr')
# driver.implicitly_wait(1)

In [ ]:
driver.switch_to.default_content()
iframe = driver.find_elements_by_tag_name('iframe')[0]
driver.switch_to.frame(iframe)

In [18]:
#driver.find_element_by_css_selector('#searchResultsDiv > table.grid > tbody > tr:nth-child(2) > td:nth-child(1) > a').click()
s = bs.BeautifulSoup(driver.page_source,'lxml')
grantInfo = s.find('div',attrs={"id":"synopsisDetailsGeneralInfoTableLeft"})

In [19]:
grantInfo

<div id="synopsisDetailsGeneralInfoTableLeft"><table border="0" cellpadding="5" cellspacing="0"><tbody><tr><th class="label" nowrap="nowrap" scope="row" valign="top">Document Type:</th><td><span class="search-custom">Grants Notice</span></td></tr><tr><th class="label" nowrap="nowrap" scope="row" valign="top">Funding Opportunity Number:</th><td><span class="search-custom">21-508</span></td></tr><tr><th class="label" nowrap="nowrap" scope="row" valign="top">Funding Opportunity Title:</th><td><span class="search-custom">Transitions to Excellence in Molecular and Cellular Biosciences Research</span></td></tr><tr><th class="label" nowrap="nowrap" scope="row" valign="top">Opportunity Category:</th><td><span class="search-custom">Discretionary</span></td></tr><tr><th class="label" nowrap="nowrap" scope="row" valign="top">Opportunity Category Explanation:</th><td><span class="search-custom"> </span></td></tr><tr><th class="label" nowrap="nowrap" scope="row" valign="top">Funding Instrument Type

In [ ]:
rows = grantInfo.find_all('tr')

In [ ]:
rows